## API source:
https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Backtest.__init__&gsc.tab=0

# What I accomplish this week

## - Applying backtesting.py, a backtesting package, on two strategies - SmaCross and Bollinger mean reversion strategy(On Bitcoin).

## - Getting acquainted with this package and knowing what this package can achieve

# Next step!

## - stop loss (by time, by loss)

## - optimization (returns, Sharpe ratio, Sortino Ratio, Calmar Ratio)

## - Applying this package to crypto-pair trading 

# Next Next step!

## - Bollinger Bands Trend Trading strategy
https://www.trendfollowingsystem.com/bollinger-bands-trend-trading-system/

# My questions
## - Is a strategy with higher Sortino Ratio definitely a greater strategy? When i am tuning my hyperparameters, Which is my target, High returns or High sharpe ratio?  Or It depends?

# My thoughts

## - I am really appreciated that Dr. Max and Mr. Sheldon give me this opportunity to participate this startup and to learn myriads of knowledge regarding trading strategy. And I am literally moved by both of your passion. I really enjoy these - programming, strategy, Linebot. I feel like i am fulfilling my dream of being a quant, which is also inspired by Netflix series 'Billions' 2 years ago. Thanks!

## - Besides, I recently meditate on what 'commitment' means to me. It means a lot ! So I reassess my progress this past two month. And i find out that with different task difficulty, the time i take to complete a task may differ. For a new or huge task likes building a Linebot, it take me two to three week. And for tasks about improving codes or adding new function to existing Linebot or strategies, it will takes me approximate one week to finish.

# Let's Coding!

# SmaCross Strategy

In [1]:
import pandas as pd


def SMA(values, n):#our indicators
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [2]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

/Users/justin_hsu/opt/anaconda3/lib/python3.8/site-packages/backtesting/_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# Loading data from binance futures api

In [3]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from binance_f import RequestClient
from binance_f.model import *
from binance_f.constant.test import *
from binance_f.base.printobject import *
import time
import threading
import math

In [4]:
# the  'api_key_secret.txt' should be a text file which record your api_key on first line and api_secret on second line

with open('api_key_secret.txt') as f:
    lines = f.read().splitlines() 
    f.close()
    

# API Key (You need to get these from Binance account)
g_api_key = lines[0]
g_api_secret = lines[1]

# connect to binance futures using api
request_client = RequestClient(api_key=g_api_key, secret_key=g_api_secret)

In [5]:

history_days = 1000
    
    
result = request_client.get_candlestick_data(symbol='BTCUSDT', interval=CandlestickInterval.DAY1, 
												startTime= None, endTime=None, limit =history_days)


[[1567900800000,"10000","10412.65","10000","10391.63","3096.291",1567987199999,"32096280.44997",3754,"0.039","393.35627","0"],[1567987200000,"10316.62","10475.54","10077.22","10307","14824.373",1568073599999,"152447209.57153",11296,"5452.451","56197395.22191","0"],[1568073600000,"10307","10382.97","9940.87","10102.02","9068.955",1568159999999,"92717645.17470",10089,"6395.462","65526356.95107","0"],[1568160000000,"10094.27","10293.11","9884.31","10159.55","10897.922",1568246399999,"109786235.83974",13811,"7724.442","77841159.69945","0"],[1568246400000,"10163.06","10450.13","10042.12","10415.13","15609.634",1568332799999,"159494237.43414",20060,"10117.798","103160660.48429","0"],[1568332800000,"10414.96","10440.55","10153.51","10341.34","19252.721",1568419199999,"198205401.80487",29249,"9717.128","100083062.05222","0"],[1568419200000,"10343.01","10419.97","10222.77","10332.25","20656.624",1568505599999,"213270510.33553",30436,"9522.547","98347059.96283","0"],[1568505600000,"10333.47","10

In [6]:
D = pd.DataFrame()

D['Open'] = [result[x].open for x in range(len(result))]
D['Close'] = [result[x].close for x in range(len(result))]
D['Low'] = [result[x].low for x in range(len(result))]
D['High'] = [result[x].high for x in range(len(result))]
D['Volume'] = [result[x].volume for x in range(len(result))]

D['Open']= D['Open'].astype(float)
D['Close']= D['Close'].astype(float)
D['Low']= D['Low'].astype(float)
D['High']= D['High'].astype(float)
D['Volume']= D['Volume'].astype(float)

## convert volume to int
D['Volume']= D['Volume'].astype(int)

D['C_l_h_mean'] = (D['Close'].astype(float)+D['Low'].astype(float)+D['High'].astype(float))/3

date_time = pd.to_datetime([dt.datetime.fromtimestamp((result[x].closeTime / 1000)-86400).strftime("%Y-%m-%d") for x in range(len(result))])
D = D.set_index(pd.DatetimeIndex(date_time.values))
D['datetime']=date_time.values

In [7]:
from backtesting import Backtest

bt = Backtest(D, SmaCross, cash=1000000, commission=.002)
stats = bt.run()
stats

Start                     2019-09-08 00:00:00
End                       2021-08-15 00:00:00
Duration                    707 days 00:00:00
Exposure Time [%]                      94.774
Equity Final [$]                  2.67255e+06
Equity Peak [$]                   7.23632e+06
Return [%]                            167.255
Buy & Hold Return [%]                 342.215
Return (Ann.) [%]                     65.9954
Volatility (Ann.) [%]                 131.057
Sharpe Ratio                         0.503563
Sortino Ratio                         1.47332
Calmar Ratio                         0.943601
Max. Drawdown [%]                    -69.9399
Avg. Drawdown [%]                    -8.11185
Max. Drawdown Duration      219 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                   35
Win Rate [%]                               40
Best Trade [%]                        182.195
Worst Trade [%]                      -19.9947
Avg. Trade [%]                    

In [8]:
bt.plot()

In [9]:
D

,Open,Close,Low,High,Volume,C_l_h_mean,datetime
2019-09-08,10000.00,10391.63,10000.00,10412.65,3096,10268.093333,2019-09-08
2019-09-09,10316.62,10307.00,10077.22,10475.54,14824,10286.586667,2019-09-09
2019-09-10,10307.00,10102.02,9940.87,10382.97,9068,10141.953333,2019-09-10
2019-09-11,10094.27,10159.55,9884.31,10293.11,10897,10112.323333,2019-09-11
2019-09-12,10163.06,10415.13,10042.12,10450.13,15609,10302.460000,2019-09-12
...,...,...,...,...,...,...,...
2021-08-11,45589.99,45522.12,45354.00,46754.27,390660,45876.796667,2021-08-11
2021-08-12,45522.13,44385.09,43800.00,46226.67,468276,44803.920000,2021-08-12
2021-08-13,44385.09,47792.52,44208.88,47980.69,438025,46660.696667,2021-08-13
2021-08-14,47792.52,47083.74,46000.00,48300.00,374368,47127.913333,2021-08-14


# Bollinger mean reversion

In [10]:
## declare our indicators

def STD(values, n):
    """
    Return simple moving std of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).std(ddof = 0)

def Upperband(mean, std, bandwidth):
    
    
    return mean + std*bandwidth

def Lowerband(mean, std, bandwidth):
    
    
    return mean - std*bandwidth

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

def Zscore(price, mean, std):
    """
    Return simple moving std of `values`, at
    each step taking into account `n` previous values.
    """
    return (price-mean)/std

In [11]:
from backtesting import Strategy
from backtesting.lib import crossover


class Bollingerband_reversion(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 30 # look back periods
    entryZ= 3 #2 # The distance between the upper and the lower band is 2*entryZ  ex. 2
    exitZ= 1 # Exit when price revert to 1 standard deviation from the mean ex.1
    pos = 0 #position, appear in later entry and exit contraint

 
    
    def init(self):
        # Precompute the two moving averages
        
        # we have to declare our indicator with the use of self.I() function. So in the next() function the 'indicator'[-1] can dynamically be updated 
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.std1 = self.I(STD, self.data.Close, self.n1)
        self.upperband = self.I(Upperband,  self.sma1 , self.std1, self.entryZ )
        self.lowerband = self.I(Lowerband,  self.sma1, self.std1, self.entryZ )
        self.zscorePrice = self.I(Zscore,  self.data.Close, self.sma1, self.std1)
        
    
        

    
    def next(self):
        
        #print(0)
        
        if (self.pos == 0) and (self.zscorePrice[-1] < (-self.entryZ)): #entry only
            #place order : market buy 1 numContract
            self.buy()
            
            '''
            result = request_client.post_order(symbol=symbol_to_trade_futures, side=OrderSide.BUY, ordertype=OrderType.MARKET, 
                                               quantity=numContract, stopPrice=None, closePosition=False)
            PrintBasic.print_obj(result)
            '''
            
            #update pos : pos=numContract
            self.pos = 1
            
        elif (self.pos < 0) and (self.zscorePrice[-1] < (-self.entryZ)): #exit and entry
            #place order : market buy 2 numContract
            self.position.close()
            self.buy()
            
            '''
            result = request_client.post_order(symbol=symbol_to_trade_futures, side=OrderSide.BUY, ordertype=OrderType.MARKET, 
                                               quantity= 2*numContract, stopPrice=None, closePosition=False)
            PrintBasic.print_obj(result)
            '''
            
            #update pos : pos=numContract
            self.pos = 1
            
        elif (self.pos < 0) and (self.zscorePrice[-1] < -self.exitZ): #exit only
            #place order : market buy 1 numContract
            self.position.close()
            
            '''
            result = request_client.post_order(symbol=symbol_to_trade_futures, side=OrderSide.BUY, ordertype=OrderType.MARKET, 
                                               quantity=numContract, stopPrice=None, closePosition=False)
            PrintBasic.print_obj(result)
            '''
            
            #update pos : pos=0
            self.pos = 0
            
        elif (self.pos == 0) and (self.zscorePrice[-1] > self.entryZ): #entry only
            #place order : market sell 1 numContract
            self.sell()
            
            '''
            result = request_client.post_order(symbol=symbol_to_trade_futures, side=OrderSide.SELL, ordertype=OrderType.MARKET, 
                                               quantity=numContract, stopPrice=None, closePosition=False)
            PrintBasic.print_obj(result)
            '''
            
            #update pos : pos=-numContract
            self.pos = -1
            
        elif (self.pos > 0) and (self.zscorePrice[-1] > self.entryZ): #exit and entry
            #place order : market sell 2 numContract
            self.position.close()
            self.sell()
            
            
            '''
            result = request_client.post_order(symbol=symbol_to_trade_futures, side=OrderSide.SELL, ordertype=OrderType.MARKET, 
                                               quantity= 2*numContract, stopPrice=None, closePosition=False)
            PrintBasic.print_obj(result)
            '''
            
            #update pos : pos=-numContract
            self.pos = -1
            
        elif (self.pos > 0) and (self.zscorePrice[-1] > self.exitZ): #exit only
            #place order : market sell 1 numContract
            self.position.close()
            
            '''
            result = request_client.post_order(symbol=symbol_to_trade_futures, side=OrderSide.SELL, ordertype=OrderType.MARKET, 
                                               quantity=numContract, stopPrice=None, closePosition=False)
            PrintBasic.print_obj(result)
            '''
            
            #update pos : pos=0
            self.pos = 0
        
        
        
        
        
        
        
        '''
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
        '''

In [12]:
bt = Backtest(D, Bollingerband_reversion, cash=1000000, commission=.002, trade_on_close=True, hedging=False, exclusive_orders=False)
stats = bt.run()
stats

Start                     2019-09-08 00:00:00
End                       2021-08-15 00:00:00
Duration                    707 days 00:00:00
Exposure Time [%]                     42.7966
Equity Final [$]                            0
Equity Peak [$]                   1.72119e+06
Return [%]                               -100
Buy & Hold Return [%]                 342.215
Return (Ann.) [%]                           0
Volatility (Ann.) [%]                 257.654
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                        -100
Avg. Drawdown [%]                    -13.6442
Max. Drawdown Duration      461 days 00:00:00
Avg. Drawdown Duration       45 days 00:00:00
# Trades                                    8
Win Rate [%]                               50
Best Trade [%]                        53.4448
Worst Trade [%]                      -107.579
Avg. Trade [%]                    

In [13]:
bt.plot()